In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display
import ipywidgets as widgets

CSV_PATH = "okashi_sample.csv"
NAME_COL = "お菓子名"
COLOR_COL = "価格[円]"

# CSV読み込み
df = pd.read_csv(CSV_PATH, encoding="shift_jis")

# 平行座標図に使う列を決める（名前列以外を数値軸として扱う）
num_cols = [c for c in df.columns if c != NAME_COL]

# 平行座標図をプロット
base_fig = px.parallel_coordinates(
    df,
    dimensions=num_cols,
    color=COLOR_COL,
    color_continuous_scale="jet",
)

# FigureWidget化
fig = go.FigureWidget(base_fig)

# 選択中候補を表示する出力領域
out = widgets.Output()

def mask_from_fig():
    """選択範囲を読み取る"""

    # 最初は全行をTrueとしてスタート
    mask = pd.Series(True, index=df.index)

    # 平行座標図の各軸(dim)と、対応する列名(num_cols)をセットで処理する
    for dim, col in zip(fig.data[0].dimensions, num_cols):

        # 選択範囲の読み取り
        cr = dim.constraintrange

        # 絞り込み設定がされていない場合 -> 何もしない
        if cr is None or cr == []:
            continue

        # 複数範囲の場合（複数範囲が結合されている場合も含む） -> [[lo,hi], [lo,hi], ...] 形式として処理
        if isinstance(cr[0], (list, tuple)):
            in_any = pd.Series(False, index=df.index)
            for lo, hi in cr:
                in_any |= df[col].between(lo, hi)

            # 他の軸条件（mask）とも両方満たす必要があるので AND で合成
            mask &= in_any

        # その他：単一範囲の場合 -> [lo, hi] 範囲内の行だけ残す（ANDで合成）
        else:
            lo, hi = cr
            mask &= df[col].between(lo, hi)
    
    return mask

def update_output(*_):
    """現在の絞り込み条件に合う候補名を Output に表示する。"""
    
    mask = mask_from_fig()
    names = df.loc[mask, NAME_COL].tolist()
    with out:
        out.clear_output()
        print(f"選択中: {len(names)} 件")
        for name in names:
            print(name)

# 絞り込み変更時に候補一覧を更新
for dim in fig.data[0].dimensions:
    dim.on_change(update_output, "constraintrange")

# 初回表示
update_output()

# 表示
display(fig, out)


FigureWidget({
    'data': [{'dimensions': [{'label': '価格[円]', 'values': {'bdata': 'lgDIAGQAeACCAMgAtAA=', 'dtype': 'i2'}},
                             {'label': '量[g]', 'values': {'bdata': 'SVYKMjc8Mg==', 'dtype': 'i1'}},
                             {'label': 'カロリー[kcal]', 'values': {'bdata': 'bQH0ATAAFgE0AVcA+gA=', 'dtype': 'i2'}},
                             {'label': '脂質[g]',
                              'values': {'bdata': 'mpmZmZmZNkBmZmZmZqZAQAAAAAAAABBAmpmZmZmZMEDNzMzMzMwzQAAAAAAAAAhAAAAAAACANEA=',
                                         'dtype': 'f8'}},
                             {'label': 'たんぱく質[g]',
                              'values': {'bdata': 'MzMzMzMzEUDNzMzMzMwhQJqZmZmZmek/ZmZmZmZmDkBmZmZmZmYGQGZmZmZmZvY/MzMzMzMzEUA=',
                                         'dtype': 'f8'}},
                             {'label': '炭水化物[g]',
                              'values': {'bdata': 'mpmZmZnZRUDNzMzMzIxEQAAAAAAAABRAzczMzMxMPEAzMzMzM7M9QAAAAAAAACtAmpmZmZmZJEA=',
       

Output()